<center>
<br />
<h1>Пространства имен, области видимости, замыкания, декораторы</h1>
<h3>Python</h3>
<br />


# Анонимные функции

In [61]:
func = lambda x : print(x)

In [62]:
func(3)

3


In [63]:
list(map(lambda x : x ** 2, range(10)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [65]:
sorted([1, 2, 3, 4], key = lambda x : -x)

[4, 3, 2, 1]

# Пространства имен [Namespaces]

Пространство имён -- мэппинг из имен переменных в объекты.

В каком-то смысле это dict, где ключи - имена объектов, а значения - ссылки на них

В коде может быть несколько пространств имён (более того, скорее всего их будет больше одного)

В частности, _внутри_ функции переменные не те же, что _снаружи_. Значит и пространство имен в ней другое


### Локальная область видимости

In [66]:
def show_scope(x):
    y = " World!"
    print(locals())  # текущее пространство имен 

show_scope("Hello")

{'x': 'Hello', 'y': ' World!'}


### Глобальная область видимости

In [67]:
def show_scope(x):
    y = " World!"
    print(globals())  # namespace глобальной области видимости. Все что доступно в коде

show_scope("Hello")

{'__name__': '__main__', '__doc__': 'Automatically created module for IPython interactive environment', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', 'func = lambda x : print(x)', 'func(3)', 'def show_scope(x):\n    y = " World!"\n    print(locals())\n\nshow_scope("Hello")', 'def show_scope(x):\n    y = " World!"\n    print(locals())  # текущее пространство имен \n\nshow_scope("Hello")', 'def show_scope(x):\n    y = " World!"\n    print(globals())  # namespace глобальной области видимости. Все что доступно в коде\n\nshow_scope("Hello")', 'locals() == globals()', 'locals() == globals()  # если мы находимся в глобальной области', 'def f():\n    in_func = 2\n\nf()\nin_func', 'def test(x):\n    print(locals())\n    print(locals() == globals())\n\ntest(1)', "global_var = 'global_var'\n\ndef test():\n    global_var = 'global_var_modified'\n    print('func  :', global_var)  

In [68]:
locals() == globals()  # если мы находимся в глобальной области

True

### Встроенная область видимости

Большая часть функций, доступных в языке, лежит в globals, но не все

In [69]:
'range' in globals()

False

In [70]:
range(3)

range(0, 3)

builtins - самый низкий уровень объектов (ниже чем globals), определенных в языке. 

Объекты в builtins это не питонячие объекты, а примитивы интерпритатора, написанные на С и доступные через имя (но об этом можно не думать)

In [71]:
'range' in dir(__builtins__)  # доступен через 2 подчеркивания т.к. не предполагается, 
                              # что программист будет туда лазить

True

### Функции создают своё пространство имён

In [72]:
def f():
    in_func = 2

f()
in_func

NameError: name 'in_func' is not defined

In [73]:
def test(x):
    print(locals())
    print(locals() == globals())

test(1)

{'x': 1}
False


In [ ]:
{}

### Циклы и условия не создают своё пространство имён

В отличии от С++, не все вложенные конструкции в питоне создают свое пространство имен (только функции, классы и генераторы)

In [76]:
for i in range(3):
    in_for = i ** 2
    pass

print(in_for)
print(i)

4
2


In [77]:
if True:
    in_if = 2
    
print(in_if)

2


### *Выражения-генераторы создают

In [80]:
i = 'Hello'

[i ** 2 for i in range(10)]
i

'Hello'

Для удобства лучше считать, что в циклах тоже создается свое пространство имен

_____

Как выполняется поиск переменных интерпритатором: (не путать с другой похожей аббревиатурой) 

## Правило LEGB

### Local

Внутри функции


### Enclosing*

Одна или несколько (до бесконечности) внешних областей видимости,  внутри которой определена функция


### Global

### Built-in

Последовательность очевидна -- просто идем по стеку вверх

In [82]:
global_var = 'global_var'

def test(): 
    local_var = 'local_var'
    print(locals())
    print('func:', global_var)  # global_var is in enclosing namespaces
    print('func:', local_var)

test()

print(global_var)
print(local_var)

{'local_var': 'local_var'}
func: global_var
func: local_var
global_var


NameError: name 'local_var' is not defined

In [84]:
global_var = 'global_var'

def test():
    global_var = 'global_var_modified'
    print('func  :', global_var)  # global_var shadows another variable with same name


test()
print('global:', global_var)

func  : global_var_modified
global: global_var


Операция присваения не имеет ничего общего с mutability, это операция над неймспейсами

Есть способы с этим справиться

### global

In [86]:
global_var = 'global_var'

def test():
    global global_var
    global_var = 'global_var_modified'
    print(global_var)  # global_var is in enclosing namespaces


print(global_var)
test()
print(global_var)


global_var
global_var_modified
global_var_modified


### Вложенные функции

In [87]:
def outer():
    outer_var = 'foo'
    
    def inner():
        inner_var = 'bar'
        print('inner:', outer_var)
        print('inner:', inner_var)

    inner()
    
    print('outer:', outer_var)
    print('outer:', inner_var)

outer()

inner: foo
inner: bar
outer: foo


NameError: name 'inner_var' is not defined

### Замечение

__Функции имеют доступ к внешним пространствам имён относительно того места где они были _определены_, а не _вызваны_ __

In [92]:
def f():
    print(it_)

def q(func):
    for it_ in range(10):
        func()
    print(it_)

q(f)

NameError: name 'it_' is not defined

In [93]:
def f():
    print(it)

for it in range(10):
    f()

0
1
2
3
4
5
6
7
8
9


### Проблема

Две функции, конкурирующие за одну и ту же переменную

In [94]:
def outer():
    var = 'v1'

    def inner():
        global var
        var = 'v1_up'
        print('inner :', var)

    inner()    
    print('outer :', var)
    
outer()
print('global:', var)

inner : v1_up
outer : v1
global: v1_up


Видим, что var попал в глобальный скоуп, но не в локальный enclosing скоуп (global идет c L сразу в G, пропуская E)

### Решение : nonlocal

Эту проблему решает ключевое слово __nonlocal__ (экспортирует объект на одну область видимости выше)

In [96]:
def outer():
    var_ = 'v1'

    def inner():
        nonlocal var_
        var_ = 'v1_up'
        print('inner :', var_)

    inner()    
    print('outer :', var_)
    
outer()
print('global:', var_)

inner : v1_up
outer : v1_up


NameError: name 'var_' is not defined

По возможности, глобальный неймспейс лучше вообще не трогать

Nonlocal был придумал в Python-3, а до этого были разные другие решения:

### Другое странное решение

Поскольку функция это объект, то в нее можно положить любой атрибут

In [97]:
def outer(): 
    outer.var = 'v1'  # нужно помнить, что outer.var для inner будет как атрибут класса а не инстанса

    def inner():
        outer.var = 'v2'
        print('inner:', outer.var)

    inner()
    
    print('outer:', outer.var)
    # return inner

outer()

print(outer.var)

inner: v2
outer: v2
v2


### Где теперь лежит переменная?

In [98]:
def outer():
    nonlocal_var = 'v1'

    def inner():
        print('inner:', nonlocal_var)

    return inner
    
f = outer()

f()

inner: v1


Функция outer() давно закончилась, но переменная из outer все равно сохранилась, хотя в момент вызова inner () функция outer() уже мертва (т.к. мы из нее вышли)

Работает это потому что

+ Доступны enclosing namespaces
+ Поиск происходит по неймспейсам, доступным во время создания функции, а не во время вызова

Для каждой функции нужно хранить состояние стэка неймспейсов, какое оно было во время создания функции

# Замыкания [Closures]
*In computer programming languages, a closure is a function together with a referencing environment of that function. A closure function is any function that uses a variable that is defined in an environment (or scope) that is external to that function, and is accessible within the function when invoked from a scope in which that free variable is not defined.*

Существования замыканий следует из правила LEGB и возможностью оперировать с функциями как обьектами.

Придумано это довольно давно в функциональных языках, в питоне реазиловано тоже довольно давно, в С++ кажется начиная с С++14 появилось

Функция похищает ссылки на объекты, которые уже умерли, заставляя их жить

In [102]:
def make_adder(x):
    def adder(y):
        return x + y
    return adder

In [103]:
add_two  = make_adder(2)  # все корректно работает, хотя 2 тепереь лежит непонятно где
add_five = make_adder(5)

add_two(7) + add_five(10)

24

In [104]:
make_adder(2)(7)

9

In [105]:
def cell(value = 10):
    def get():
        return value

    def set(new_value):
        nonlocal value
        value = new_value
        return value
    
    value = 20
    return get, set

get, set = cell()  # две функции, неявно связанные друг с другом через общий контекст
print(get())

set(20000000000000)

print(get())

20
20000000000000


In [107]:
def make_adder(x):
    
    def add(y):
        return x + y
    
    def update(new_x):
        nonlocal x
        x = new_x

    add.update = update
    return add

adder = make_adder(10)
print(adder(10))

adder.update(100)  # функция, у которой есть метод, меняющий ее поведение

print(adder(10))

20
110


### Посмотрим внутрь

При выходе из make_adder его неймспейс вместо того, чтобы умереть, перекладывается в __ closure __ того, что из него вернулось

In [109]:
print(adder.__closure__)
print(repr(adder.__closure__[0].cell_contents))

(<cell at 0x7f9d5062e5e0: int object at 0x10ee105c0>,)
100


In [110]:
adder.update.__closure__ == adder.__closure__ 

True

# Декораторы

Замыкания как способ быстро изменить поведение функции

In [33]:
import sys
def deprecate(func):
    def inner(*args, **kwargs):
        print('{} is deprecated'.format(func.__name__), file=sys.stderr)
        return func(*args, **kwargs)
    return inner

pprint = deprecate(print)

pprint([1, 2, 3])

[1, 2, 3]


print is deprecated


### Синтаксис декораторов

In [35]:
import sys

def deprecated(func):
    def wrapper(*args, **kwargs):
        print('{} is deprecated'.format(func.__name__), file=sys.stderr)
        return func(*args, **kwargs)
    return wrapper

@deprecated
def add(x, y):
    return x + y

add(1, 2)

add is deprecated


3

In [111]:
add.__name__

'wrapper'

Функция объявляется, создается, ее объект подается декоратору, декоратор что-то возвращает и эт что-то перезаписывается вместо функции

### Bananize

Из-за динамической типизации никто не проверяет реализацию декораторов, если что-то не так, то все просто упадет

In [113]:
from IPython import display

def bananize(func):
    return display.HTML('<img src="http://www.sherv.net/cm/emo/funny/2/big-dancing-banana-smiley-emoticon.gif">')

@bananize
def show(x):
    print(x)
    
show

### Проблема

In [114]:
@deprecated
def show(x):
    'This is a really nice looking docstring'
    print(x)

print(show.__name__)
print(show.__doc__)

wrapper
None


Действительно, это теперь не show, а то что вернул декоратор

### Решение 1 (грубой силой)

In [115]:
def deprecated(func):
    def wrapper(*args, **kwargs):
        print('{} is deprecated!'.format(func.__name__), file=sys.stderr)
        return func(*args, **kwargs)
    wrapper.__name__ = func.__name__
    wrapper.__doc__ = func.__doc__
    wrapper.__module__ = func.__module__
    return wrapper

@deprecated
def show(x):
    'This is a really nice looking docstring'
    print(x)

print(show.__name__)
print(show.__doc__)

show
This is a really nice looking docstring


### Решение 2

Используем декоратор на wrapper

In [116]:
import functools

def deprecated(func):
    @functools.wraps(func) 
    def wrapper(*args, **kwargs):
        print('{} is deprecated!'.format(func.__name__), file=sys.stderr)
        return func(*args, **kwargs)
    return wrapper

@deprecated
def show(x):
    'This is a really nice looking docstring'
    print(x)

print(show.__name__)
print(show.__doc__)

show
This is a really nice looking docstring


Заметим, что до этого декоратор был штукой, принимающей функцию и возвращающей функцию

Сейчас это функция, которая возвращает функцию, которая возвращает функцию и принимает функцию


<img src='http://i0.kym-cdn.com/photos/images/original/000/384/176/d2f.jpg' />

In [120]:
import functools 


def trace(dest=sys.stderr):
    def wraps(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print(
                '{} called with args {}, kwargs {}!'.format(func.__name__, args, kwargs),
                file = dest
            )
            return func(*args, **kwargs)
        return wrapper
    return wraps

@trace()  # нужно теперь писать скобки, даже если ничего не подавать
def f(x, test):
     if test > 1:
        return f(x, test / 2)

f('Hi!', test=42)

f called with args ('Hi!',), kwargs {'test': 42}!
f called with args ('Hi!', 21.0), kwargs {}!
f called with args ('Hi!', 10.5), kwargs {}!
f called with args ('Hi!', 5.25), kwargs {}!
f called with args ('Hi!', 2.625), kwargs {}!
f called with args ('Hi!', 1.3125), kwargs {}!
f called with args ('Hi!', 0.65625), kwargs {}!


### Практика :: Декоратор Once

In [121]:
def once(func):
    called = False
    
    def wrapper(*args, **kwargs):
        nonlocal called
        if not called:
            called = True
            return func(*args, **kwargs)

    return wrapper

@once
def f():
    print('Hi!')

f()
f()

Hi!


### Цепочки декораторов

Работают снизу вверх

In [122]:
@deprecated
@trace(sys.stdout)
def f(x):
    return x

f(1)
f(2)

f called with args (1,), kwargs {}!
f called with args (2,), kwargs {}!


f is deprecated!
f is deprecated!


2